In [5]:
from __future__ import division
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [6]:
building_SF = pd.read_csv('../inputs/building_block.csv')

In [3]:
building_SF_work=building_SF[['objname','centerlat', 'centerlon','num_floor','fid_landus','blklot', 'block_num', 'lot_num', 
                              'resunits', 'landuse','area_m2','tractce10', 'blockid10', 'housing10','pop10']]

In [4]:
building_SF_work.head()

,objname,centerlat,centerlon,num_floor,fid_landus,blklot,block_num,lot_num,resunits,landuse,area_m2,tractce10,blockid10,housing10,pop10
0,SanfranQ_1047,37.751639,-122.479761,5,49819,2112001,2112,001,1,RESIDENT,130.614398,32802,6.075033e+13,62,146
1,SanfranQ_1047,37.751639,-122.479761,5,49820,2112001A,2112,001A,1,RESIDENT,116.067280,32802,6.075033e+13,62,146
2,SanfranQ_1047,37.751639,-122.479761,5,51786,2112001B,2112,001B,1,RESIDENT,117.381949,32802,6.075033e+13,62,146
3,SanfranQ_1047,37.751639,-122.479761,5,52023,2112009,2112,009,1,RESIDENT,100.114528,32802,6.075033e+13,62,146
4,SanfranQ_1047,37.751639,-122.479761,5,52353,2112001D,2112,001D,1,MIXRES,156.518242,32802,6.075033e+13,62,146


In [5]:
building_SF_grouped = building_SF_work.groupby('blockid10')

In [6]:
conn = psycopg2.connect("dbname='population_SF' user='Olga' host='localhost'")
cur = conn.cursor()


In [7]:
#Edit database for floor 1
cur.execute(""" 
UPDATE  building_point_block
SET num_floor = 1
WHERE num_floor = 0;
""")

In [8]:
#Find all buildings listed as residential and Estimate total area by using num_floors and footprint area
cur.execute(""" DROP TABLE IF EXISTS all_building_blocks;
CREATE TABLE all_building_blocks AS
SELECT *, (a.area_m2*a.num_floor) as total_area 
FROM building_point_block a
WHERE a.landuse = 'RESIDENT' OR a.landuse = 'MIXRES' OR a.landuse = 'MISSING DATA';
""")

In [9]:
#Estimate total area of residential building by BLOCK
cur.execute(""" DROP TABLE IF EXISTS sum_area_block;
CREATE TABLE sum_area_block AS
SELECT a.blockid10, SUM(a.total_area) as total_area_floors 
FROM all_building_blocks a
GROUP BY a.blockid10;
""")


In [10]:
#Estimate percent of building area for every BLOCK
cur.execute(""" DROP TABLE IF EXISTS percent_area_block;
CREATE TABLE percent_area_block AS
SELECT b.gid, a.blockid10, b.total_area/a.total_area_floors as percent_area
FROM all_building_blocks b, sum_area_block a
WHERE a.blockid10 = b.blockid10;
""")

In [15]:
#Estimate number of people by building
cur.execute(""" DROP TABLE IF EXISTS num_people_build;
CREATE TABLE num_people_build AS
SELECT b.gid, a.blockid10, a.percent_area*b.pop10 as num_people
FROM all_building_blocks b, percent_area_block a
WHERE a.gid = b.gid;
""")

In [22]:
cur.execute("""
SELECT * FROM num_people_build ;
""")
colnames = [desc[0] for desc in cur.description]
rows = cur.fetchall()

df_num_people_build = pd.DataFrame(rows)
df_num_people_build.columns=colnames

In [23]:
cur.execute("""
SELECT * FROM all_building_blocks ;
""")
colnames = [desc[0] for desc in cur.description]
rows = cur.fetchall()

df_all_building_blocks = pd.DataFrame(rows)
df_all_building_blocks.columns=colnames

In [24]:
df_all_building_blocks.head()

,gid,join_count,target_fid,fid_buildi,objname,centerlat,centerlon,origin84,originmsl,usngpos,...,fid_tabblo,statefp10,countyfp10,tractce10,blockce,blockid10,partflg,housing10,pop10,total_area
0,1,1,1,0,SanfranQ_1047,37.7516388900,-122.479760690,123.249094030,151.358031620,10SEG4582978387,...,528925,06,075,032802,1006,060750328021006,N,62,146,653.071988575
1,2,1,2,0,SanfranQ_1047,37.7516388900,-122.479760690,123.249094030,151.358031620,10SEG4582978387,...,528925,06,075,032802,1006,060750328021006,N,62,146,580.336401645
2,3,1,3,0,SanfranQ_1047,37.7516388900,-122.479760690,123.249094030,151.358031620,10SEG4582978387,...,528925,06,075,032802,1006,060750328021006,N,62,146,586.909745395
3,4,1,4,0,SanfranQ_1047,37.7516388900,-122.479760690,123.249094030,151.358031620,10SEG4582978387,...,528925,06,075,032802,1006,060750328021006,N,62,146,500.572640550
4,5,1,5,0,SanfranQ_1047,37.7516388900,-122.479760690,123.249094030,151.358031620,10SEG4582978387,...,528925,06,075,032802,1006,060750328021006,N,62,146,782.591209070


In [25]:
df_all_building_blocks.to_csv("C:\Users\Olga\Documents\RENUWIT\Urine_separation\python_model\\building_block.csv")

In [26]:
building_population=pd.merge(df_all_building_blocks, df_num_people_build, how='left', on=['gid', 'gid'])

In [28]:
building_population.to_csv("C:\Users\Olga\Documents\RENUWIT\Urine_separation\python_model\\building_population.csv")

In [34]:
a = np.zeros(4)[None].T

In [52]:
a=np.array([1,2])
b=np.array([[5,6],[7,8]])